In [294]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from scipy import stats

import warnings
from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

$X= ``Número\ de\ gols\ do\ time\ de\ casa" \sim Poisson(\lambda_x)\\$
$Y= ``Número\ de\ gols\ do\ time\ visitante" \sim Poisson(\lambda_x)$

$$P(X=x)=\frac{\lambda^{x}_{x}e^{-\lambda_x}}{x!}\qquad P(Y=y)=\frac{\lambda^{y}_{y}e^{-\lambda_y}}{y!}\\$$

$$P(X=x, Y=y)=P(X=x)P(Y=y)=\frac{\lambda_{x}^{x}\lambda_{y}^{y}e^{-(\lambda_x+\lambda_y)}}{x!y!}$$

In [295]:
tabela = pd.read_excel('dados/tabela/tabela-rodada_27.xlsx')
tabela_casa = pd.read_excel('dados/tabela/tabela_casa-rodada_27.xlsx')
historico_casa = pd.read_excel('dados/historico/historicocasa-rodada_27.xlsx')
historico = pd.read_excel('dados/historico/historico-rodada_27.xlsx')
rodadas = pd.read_excel('dados/rodadas/todas_rodadas.xlsx')
rodadas_futuras = pd.read_excel('dados/rodadas/rodadas_futuras-rodada-28.xlsx')

rodada_atual = tabela['MP'].max()

In [296]:
def gerar_lambda(par_x, par_y, peso_x, peso_y, fator_casa, rodada):
  df = rodadas_futuras[rodadas_futuras['Wk'] == rodada]
  dem_x = (sum(peso_x[0])+sum(peso_x[1]))
  dem_y = (sum(peso_y[0])+sum(peso_y[1]))

  for time_casa, time_fora in zip(df['Home'], df['Away']):
    # Time alvo == Casa
    att_x = sum(tabela.loc[tabela['Squad'] == time_casa, par_x[0][i]]*peso_x[0][i] for i in range(len(par_x[0])))/dem_x
    def_y = sum(tabela.loc[tabela['Squad'] == time_casa, par_x[1][i]]*peso_x[1][i] for i in range(len(par_x[1])))/dem_x
    casa_x = ((1/int(tabela_casa.loc[tabela_casa['Squad'] == time_casa, fator_casa[0]]))*0.5)+1
    lambda_x = att_x*def_y*casa_x
    # Time alvo == Casa
    att_y = sum(tabela.loc[tabela['Squad'] == time_casa, par_y[0][i]]*peso_y[0][i] for i in range(len(par_y[0])))/dem_y
    def_x = sum(tabela.loc[tabela['Squad'] == time_casa, par_y[1][i]]*peso_y[1][i] for i in range(len(par_y[1])))/dem_y
    lambda_y = att_y*def_x
    
    df.loc[df['Home'] == time_casa, 'lambda_x'] = float(lambda_x)
    df.loc[df['Away'] == time_fora, 'lambda_y'] = float(lambda_y)

  return df

In [297]:
def prob_vitoria(rodada_df, rodada, lambdas):
  rodada_df = rodada_df.loc[rodada_df['Wk'] == rodada]
  prob_gol = {}
  times_casa = []
  times_fora = []
  
  # Times casa
  for time in rodada_df['Home']:
    prob_gol[time] = []
    times_casa.append(time)
    for k in range(0, 6):
      prob_gol[time].append(stats.poisson.pmf(k, lambdas.loc[lambdas['Home'] == time, 'lambda_x']))
  
  # Times Fora
  for time in rodada_df['Away']:
    prob_gol[time] = []
    times_fora.append(time)
    for k in range(0, 6):
      prob_gol[time].append(stats.poisson.pmf(k, lambdas.loc[lambdas['Away'] == time, 'lambda_y']))

  casa_vitoria = {}
  fora_vitoria = {}

  for time_casa, time_fora in zip(times_casa, times_fora): 
    casa_vitoria[time_casa] = []
    fora_vitoria[time_fora] = []
    for i in range(1, 6):
      casa_vitoria[time_casa].append(prob_gol[time_casa][i] * sum(prob_gol[time_fora][:i]))
      fora_vitoria[time_fora].append(prob_gol[time_fora][i] * sum(prob_gol[time_casa][:i]))
    casa_vitoria[time_casa] = sum(casa_vitoria[time_casa])
    fora_vitoria[time_fora] = sum(fora_vitoria[time_fora])
    rodada_df.loc[rodada_df['Home'] == time_casa, 'vitoria_casa'] = casa_vitoria[time_casa]
    rodada_df.loc[rodada_df['Away'] == time_fora, 'vitoria_fora'] = fora_vitoria[time_fora]
    rodada_df.loc[rodada_df['Away'] == time_fora, 'empate'] = 1- (rodada_df.loc[rodada_df['Home'] == time_casa, 'vitoria_casa'] + rodada_df.loc[rodada_df['Away'] == time_fora, 'vitoria_fora'])
    

  return rodada_df, prob_gol

In [298]:
def prob_gols(rodadas_prob, prob_gol):
  rodadas_probgols = pd.DataFrame.from_dict(prob_gol, orient='index').astype(float)

  df_casa = pd.DataFrame()
  for time in rodadas_prob['Home']:
    linha_casa = pd.DataFrame(rodadas_probgols.loc[time,]).T
    linha_casa['Home'] = time
    df_casa = pd.concat([df_casa, linha_casa])

  df_fora = pd.DataFrame()
  for time in rodadas_prob['Away']:
    linha_fora = pd.DataFrame(rodadas_probgols.loc[time,]).T
    linha_fora['Away'] = time
    df_fora = pd.concat([df_fora, linha_fora])


  df_qsefinal = pd.merge(rodadas_prob, df_casa, on='Home')
  df_qsefinal = pd.merge(df_qsefinal, df_fora, on='Away')
  df_final = pd.merge(rodadas_futuras, df_qsefinal, on= ['Wk', 'Day', 'Date', 'Home', 'xGCasa', 'gols_casa', 'gols_fora', 'xGFora', 'Away'], how='left')


  return df_final, df_qsefinal

In [299]:
par_x = [['xG_total', 'xG_casa', 'GF_total', 'GF_casa', 'xG_5total', 'xG_5casa', 'GF_5total', 'GF_5casa'], #Ataque casa
        ['GA_total', 'GA_fora', 'GA_5total', 'GA_5fora', 'xG_5sofrido_fora', 'xG_sofrido_fora']]  #Defesa fora
pes_x = [[1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]


par_y = [['xG_total', 'xG_fora', 'GF_total', 'GF_fora', 'xG_5total', 'xG_5fora', 'GF_5total', 'GF_5fora'], #Ataque fora
        ['GA_total', 'GA_casa', 'GA_5total', 'GA_5casa', 'xG_sofrido_casa', 'xG_5sofrido_casa']]  #Defesa casa
pes_y = [[1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]

fator_casa = ['Rk_casa']

In [300]:
lambdas = gerar_lambda(par_x, par_y, pes_x, pes_y, fator_casa, rodada_atual+1)

In [301]:
rodadas_prob, prob_gol = prob_vitoria(rodadas_futuras, rodada_atual+1, lambdas)

In [303]:
rodadas_futuras_prob, rodada_futura_prob = prob_gols(rodadas_prob, prob_gol)

In [305]:
todas_previsoes = pd.read_excel('dados/previsoes/todas_previsoes.xlsx', index_col= None)
todas_previsoes.drop(todas_previsoes.columns[0], axis=1, inplace= True)

In [307]:
if (todas_previsoes[todas_previsoes['Wk'] == rodada_atual+1]['vitoria_casa'] == None).all():
  todas_previsoes = pd.concat([todas_previsoes, rodada_futura_prob], ignore_index= True)
if (todas_previsoes[todas_previsoes['Wk'] == rodada_atual]['gols_casa'] == None).all():
  resultados = rodadas[rodadas['Wk'] >= rodada_atual].reset_index().drop('index', axis= 1)
  todas_previsoes.update(resultados)

In [315]:
def compare_columns(row):
    if row['gols_casa'] > row['gols_fora']:
        return 'gols_casa'
    elif row['gols_casa'] < row['gols_fora']:
        return 'gols_fora'
    else:
        return 'empate'  # Handle cases when values are equal

def compare_columns1(row):
    if row['vitoria_casa'] > row['empate'] and row['vitoria_casa'] > row['vitoria_fora']:
        return 'gols_casa'
    elif row['vitoria_fora'] > row['empate'] and row['vitoria_fora'] > row['vitoria_casa']:
        return 'gols_fora'
    else:
        return 'empate'  # Handle cases when values are equal

def compare_columns2(row):
    if row['acerto1'] == row['acerto2']:
        return 'acerto'
    else:
        return 'fracasso'

# Apply the custom function to create the 'Larger_Column' column
todas_previsoes['acerto1'] = todas_previsoes.apply(compare_columns, axis=1)
todas_previsoes['acerto2'] = todas_previsoes.apply(compare_columns1, axis=1)
todas_previsoes['acerto'] = todas_previsoes.apply(compare_columns2, axis=1)

todas_previsoes.drop(['acerto1', 'acerto2'], axis=1, inplace= True)

In [317]:
todas_previsoes.to_excel('dados/previsoes/todas_previsoes.xlsx')
rodada_futura_prob.to_excel(f'dados/previsoes/previsao_rodada-{rodada_atual+1}.xlsx')